In [15]:
import time
import torch
from utils.image_utils import gen_index_file
from unet.dataset import SegThorImagesDataset
from torch.utils.data import DataLoader, random_split
from unet.unet_model import UNet
from unet import simplified_unet_model
from unet import loss

from importlib import reload

reload(loss)
reload(simplified_unet_model)


from torch import optim
from torch import nn
import os 
import pandas as pd 
from utils.model_utils import get_model_from_checkpoint



In [16]:
test_idx_file = gen_index_file(root='/home/jupyter/ecs271_data/data/train')

Filename: data/train_patient_idx.csv already exists, skipping gen


In [17]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. ")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead.")

DATA_DIR = '/home/jupyter/ecs271_data/data/train'
EXPERIMENT_DIR = '/home/jupyter/ecs271_data/experiments'
TRAIN_CSV = "data/train_patient_idx_sorted.csv"
VALID_CSV = "data/valid_patient_idx_sorted.csv"
INPUT_DATA_INDEX = 'data/train_patient_idx.csv'
TEST_CSV = "data/test_patient_idx.csv"

def evaluate_model(
    model,
    test_csv = TEST_CSV,
    data_dir: str = DATA_DIR):    

    test_dataset = SegThorImagesDataset(
        patient_idx_file=test_csv,
        root_dir=data_dir,
        img_crop_size=312,
        mask_output_size=220,
        cache_size=1
    )
    test_dl = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    criterion = loss.GeneralizedDiceLoss(classes=[0, 1, 2, 3, 4])
    
    model.to(device)
    model.eval()
    class_losses = []
    with torch.no_grad():
        for idx, sample in enumerate(test_dl):
            start_time = time.time()
            inputs, targets = sample
            inputs=inputs.to(device)
            targets=targets.to(device)
            outputs=model.predict_class_channels(inputs)
            # want this per channel
            per_channel_dice = criterion.dice_per_channel(outputs, targets.long())
            per_channel_dice = per_channel_dice.to('cpu')
            class_losses.append(per_channel_dice)
    return torch.cat(class_losses, dim=0)
    

GPU is available. 


In [18]:
from utils.model_utils import get_model_from_checkpoint

experiment_name = 'simplified_unet_08_limited_train'
model_epoch = simplified_unet_model.SimplifiedUNet(n_channels=1, n_classes=5, dropout=0.2)
# model_epoch = get_model_from_checkpoint(experiment_dir = EXPERIMENT_DIR, 
#                                         experiment_name = experiment_name, 
#                                         model=model_epoch, 
#                                         epoch=39)

In [19]:
channel_dice = evaluate_model(model_epoch)

In [20]:
cat_means = channel_dice.mean(dim=0)

In [21]:
loss.print_dice_by_category(cat_means)

background: 0.4275, esophagus: 0.0040, heart: 0.0262, trachea: 0.0006, aorta: 0.0024


In [8]:
loss.print_dice_by_category(cat_means)

background: 0.9740, esophagus: 0.4946, heart: 0.4301, trachea: 0.7526, aorta: 0.6907


In [ ]:
from utils import image_utils
from importlib import reload
reload(image_utils)

# get sample 
test_dataset = SegThorImagesDataset(
        patient_idx_file=TEST_CSV,
        root_dir=DATA_DIR,
        img_crop_size=312,
        mask_output_size=220,
        cache_size=1
)

X, Y = test_dataset[350]
X = X.to(device)
Y = Y.to(device)

image_utils.plot_XY_pred_class(model_epoch, X, Y)